In [1]:
pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pandas (from versions: none)
ERROR: No matching distribution found for pandas

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 660.6 kB/s eta 0:00:03
   - -------------------------------------- 0.1/1.8 MB 544.7 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/1.8 MB 1.1 MB/s eta 0:00:02
   ------ --------------------------------- 0.3/1.8 MB 1.7 MB/s eta 0:00:01
   ---------- ----------------------------- 0.5/1.8 MB 2.2 MB/s eta 0:00:01
   ------------- -------------------------- 0.6/1.8 MB 2.3 MB/s eta 0:00:01
   ----------------- ---------------------- 0.8/1.8 MB 2.6 MB/s eta 0:00:01
   --------------------- ------------------ 1.0/1.8 MB 2.7 MB/s eta 0:00:01
   ------------------------ --------------- 1.1/1.8 MB 2.8 MB/s eta 0:00:01
   ---------------------------- ----------- 1.3/1.8 MB 2.9 MB/s eta 0:00:01
   ------------------------------- -------- 1.4/1.8 MB 3.0 MB/s eta 0:00:01
   -------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [8]:
import pandas as pd
import numpy as np

def load_data(file_path):
    """Load keylogging data from a CSV file."""
    data = pd.read_csv(file_path)
    return data

def extract_mouse_features(data):
    """Extract features from mouse movements."""
    # Filter mouse move events
    mouse_moves = data[data['Event'] == 'MouseMove']
    
    # Extract mouse coordinates from 'Key/Mouse' column
    mouse_moves['X'] = mouse_moves['Key/Mouse'].apply(lambda x: int(x.split('(')[1].split(',')[0]))
    mouse_moves['Y'] = mouse_moves['Key/Mouse'].apply(lambda x: int(x.split(',')[1].split(')')[0]))
    
    # Calculate distance between consecutive points
    mouse_moves['dX'] = mouse_moves['X'].diff().fillna(0)
    mouse_moves['dY'] = mouse_moves['Y'].diff().fillna(0)
    mouse_moves['distance'] = np.sqrt(mouse_moves['dX']**2 + mouse_moves['dY']**2)
    
    # Calculate time difference between consecutive points
    mouse_moves['time_diff'] = mouse_moves['Timestamp'].diff().fillna(0)
    
    # Calculate speed (distance/time)
    mouse_moves['speed'] = mouse_moves['distance'] / mouse_moves['time_diff']
    mouse_moves['speed'].replace([np.inf, -np.inf], 0, inplace=True)  # Replace inf values with 0

    return mouse_moves[['distance', 'speed', 'time_diff']]

def extract_key_features(data):
    """Extract features from key events."""
    key_events = data[data['Event'] == 'KeyDown']
    
    # Calculate time between consecutive key presses
    key_events['time_diff'] = key_events['Timestamp'].diff().fillna(0)
    
    # Calculate frequency of each key pressed
    key_frequency = key_events['Key/Mouse'].value_counts().reset_index()
    key_frequency.columns = ['Key', 'Frequency']
    
    return key_events[['Key/Mouse', 'time_diff']], key_frequency

def prepare_features(session_1_data, session_2_data):
    """Prepare features from both sessions."""
    # Extract mouse and key features for both sessions
    mouse_features_1 = extract_mouse_features(session_1_data)
    key_features_1, key_frequency_1 = extract_key_features(session_1_data)
    
    mouse_features_2 = extract_mouse_features(session_2_data)
    key_features_2, key_frequency_2 = extract_key_features(session_2_data)

    return mouse_features_1, key_features_1, key_frequency_1, mouse_features_2, key_features_2, key_frequency_2

# Example usage:
session_1_path = 'C:\Users\unico\Desktop\amrita EAC sem 7\cs\Project\dataset\Virinchi_Session_1_dup.csv'
session_2_path = 'C:\Users\unico\Desktop\amrita EAC sem 7\cs\Project\dataset\Virinchi_Session_1.csv'

# Load data
session_1_data = load_data(session_1_path)
session_2_data = load_data(session_2_path)

# Prepare features
mouse_features_1, key_features_1, key_frequency_1, mouse_features_2, key_features_2, key_frequency_2 = prepare_features(session_1_data, session_2_data)

print("Mouse features from Session 1:\n", mouse_features_1.head())
print("Key features from Session 1:\n", key_features_1.head())
print("Key frequency from Session 1:\n", key_frequency_1.head())


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1894434570.py, line 57)